In [1]:
from pathlib import Path
from torch.utils.data import Dataset
import numpy as np
import pickle
import tqdm
import torch
from torch.utils.data import DataLoader
from tool.dataset import clocs_data

/home/drcl/anaconda3/envs/mmlab/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_CUDA_DRIVER=/usr/lib/x86_64-linux-gnu/libcuda.so.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/drcl/anaconda3/envs/mmlab/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda-11.3/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/drcl/anaconda3/envs/mmlab/lib/python3.8/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variab

In [4]:
def build_stage2_training(boxes, query_boxes, criterion, scores_3d, scores_2d, dis_to_lidar_3d,overlaps,tensor_index):
    N = boxes.shape[0] #20000
    print(N)
    K = query_boxes.shape[0] #30
    print(K)
    max_num = 900000
    ind=0
    ind_max = ind
    for k in range(K):
        qbox_area = ((query_boxes[k, 2] - query_boxes[k, 0]) *
                     (query_boxes[k, 3] - query_boxes[k, 1]))
        for n in range(N):
            iw = (min(boxes[n, 2], query_boxes[k, 2]) -
                  max(boxes[n, 0], query_boxes[k, 0]))
            if iw > 0:
                ih = (min(boxes[n, 3], query_boxes[k, 3]) -
                      max(boxes[n, 1], query_boxes[k, 1]))
                if ih > 0:
                    if criterion == -1:
                        ua = (
                            (boxes[n, 2] - boxes[n, 0]) *
                            (boxes[n, 3] - boxes[n, 1]) + qbox_area - iw * ih)
                    elif criterion == 0:
                        ua = ((boxes[n, 2] - boxes[n, 0]) *
                              (boxes[n, 3] - boxes[n, 1]))
                    elif criterion == 1:
                        ua = qbox_area
                    else:
                        ua = 1.0
                    overlaps[ind,0] = iw * ih / ua #IOU
                    overlaps[ind,1] = scores_3d[n]
                    overlaps[ind,2] = scores_2d[k,0]
                    overlaps[ind,3] = dis_to_lidar_3d[n,0]
                    tensor_index[ind,0] = k
                    tensor_index[ind,1] = n
                    ind = ind+1

                elif k==K-1:
                    overlaps[ind,0] = -10
                    overlaps[ind,1] = scores_3d[n]
                    overlaps[ind,2] = -10
                    overlaps[ind,3] = dis_to_lidar_3d[n,0]
                    tensor_index[ind,0] = k
                    tensor_index[ind,1] = n
                    ind = ind+1
            elif k==K-1:
                overlaps[ind,0] = -10
                overlaps[ind,1] = scores_3d[n]
                overlaps[ind,2] = -10
                overlaps[ind,3] = dis_to_lidar_3d[n,0]
                tensor_index[ind,0] = k
                tensor_index[ind,1] = n
                ind = ind+1
    if ind > ind_max:
        ind_max = ind
    return overlaps, tensor_index, ind

In [5]:
class clocs_data(Dataset):

    def __init__(self, _2d_path, _3d_path, index_path, input_data, infopath = '../data/clocs_data/info/kitti_infos_trainval.pkl', val = False):

        self._2d_path = _2d_path 
        self._3d_path = _3d_path 
        f = open(index_path, "r")
        self.ind = f.read().splitlines()
        f.close()
        self.val = val #False
        self.input_data = input_data #input data path
        self.anno = pickle.load(open(infopath,'rb'))
        self.id2ind = np.zeros(len(self.anno))
        # self.id2ind[img_id] = number in self.anno
        for i in range(len(self.anno)):
            # print(self.anno[i])
            self.id2ind[int(self.anno[i]['image_idx'])] = i

    def generate_input(self):
        input_path = Path(self.input_data)

        # for i in tqdm(range(len(self.ind))):
        for i in range(1):
            
            i = 7

            # find 2d detection
            detection_2d_file_name = self._2d_path+"/"+self.ind[i]+".txt"
            print("detection_2d_file_name :",detection_2d_file_name)

            with open(detection_2d_file_name, 'r') as f:
                lines = f.readlines()
            content = [line.strip().split(' ') for line in lines]

            # print("content : \n",content)

            predicted_class = np.array([x[0] for x in content],dtype='object')
            predicted_class_index = np.where(predicted_class=='Car')

            detection_result = np.array([[float(info) for info in x[4:8]] for x in content]).reshape(-1, 4)
            score = np.array([float(x[15]) for x in content])  # 1000 is the score scale!!!

            f_detection_result=np.append(detection_result,score.reshape(-1,1),1)

            # print("f_detection_result \n", f_detection_result)
            # print(f_detection_result.shape)

            middle_predictions=f_detection_result[predicted_class_index,:].reshape(-1,5)

            # print("middle_predictions \n", middle_predictions)
            # print(middle_predictions.shape)

            top_predictions=middle_predictions[np.where(middle_predictions[:,4]>=-100)]

            # print("top_predictions \n", top_predictions)
            # print(top_predictions.shape)

            _3d_result = torch.load(self._3d_path+"/"+self.ind[i]+".pt")[0]

            # print("_3d_result \n", _3d_result)

            res, iou_test, tensor_index = self.train_stage_2(_3d_result, top_predictions)

            all_3d_output_camera_dict, fusion_input,tensor_index = res, iou_test, tensor_index

            # get 3d anno
            int_ind = int(self.id2ind[i])
            gt_anno = self.anno[int_ind]['annos']
            d3_gt_boxes = self.process_anno(gt_anno)

            d3_gt_boxes_camera=d3_gt_boxes
            if d3_gt_boxes.shape[0] == 0:
                target_for_fusion = np.zeros((1,20000,1))
                positive_index = np.zeros((1,20000),dtype=np.float32)
                negative_index = np.zeros((1,20000),dtype=np.float32)
                negative_index[:,:] = 1
                
            else:
                ###### predicted bev boxes
                pred_3d_box = all_3d_output_camera_dict[0]["box3d_camera"]

                print("d3_gt_boxes_camera \n",d3_gt_boxes_camera)
                print("pred_3d_box \n",pred_3d_box)


                iou_bev = d3_box_overlap(d3_gt_boxes_camera, pred_3d_box, criterion=-1)
                iou_bev_max = np.amax(iou_bev,axis=0)
                # print(np.max(iou_bev_max))
                target_for_fusion = ((iou_bev_max >= 0.7)*1).reshape(1,-1,1)

                positive_index = ((iou_bev_max >= 0.7)*1).reshape(1,-1)
                negative_index = ((iou_bev_max <= 0.5)*1).reshape(1,-1)
            
    def train_stage_2(self, _3d_result,top_predictions):
        box_preds = _3d_result['boxes_lidar']
        final_box_preds = box_preds
        predictions_dicts = []
        locs = _3d_result['location'] # xyz
        dims = _3d_result['dimensions'] # hwl
        angles = _3d_result['rotation_y'].reshape(-1,1)
        final_box_preds_camera = np.concatenate((locs,dims,angles),axis=1)

        # print("final_box_preds_camera \n",final_box_preds_camera)
        print("final_box_preds \n",final_box_preds)


        box_2d_preds = _3d_result['bbox']
        final_scores = _3d_result['score']
        img_idx = _3d_result['frame_id']
        # predictions
        predictions_dict = {
            "bbox": box_2d_preds,
            "box3d_camera": final_box_preds_camera,
            "box3d_lidar": final_box_preds,
            "scores": final_scores,
            #"label_preds": label_preds,
            "image_idx": img_idx,
        }

        predictions_dicts.append(predictions_dict)
        # normalization 된 라이다와 박스 사이의 직선 거리   
        dis_to_lidar = torch.norm(torch.tensor(box_preds[:,:2]),p=2,dim=1,keepdim=True).numpy()/82.0 

        box_2d_detector = np.zeros((200, 4))
        box_2d_detector[0:top_predictions.shape[0],:]=top_predictions[:,:4]
        box_2d_detector = top_predictions[:,:4]
        box_2d_scores = top_predictions[:,4].reshape(-1,1)
        overlaps1 = np.zeros((900000,4),dtype=np.float32)
        tensor_index1 = np.zeros((900000,2),dtype=np.float32)
        overlaps1[:,:] = -1.0
        tensor_index1[:,:] = -1.0


        iou_test,tensor_index, max_num = build_stage2_training(box_2d_preds,
                                    box_2d_detector,
                                    -1,
                                    final_scores,
                                    box_2d_scores,
                                    dis_to_lidar,
                                    overlaps1,
                                    tensor_index1)
        print("box_2d_detector \n", box_2d_detector)
        print("iou_test \n", iou_test[:max_num])
        print("tensor_index \n", tensor_index[:max_num])
        print("max_num\n", max_num)

        iou_test_tensor = torch.FloatTensor(iou_test)  #iou_test_tensor shape: [160000,4]
        tensor_index_tensor = torch.LongTensor(tensor_index)
        iou_test_tensor = iou_test_tensor.permute(1,0)

        iou_test_tensor = iou_test_tensor.reshape(4,900000)

        tensor_index_tensor = tensor_index_tensor.reshape(-1,2)

        if max_num == 0:
            non_empty_iou_test_tensor = torch.zeros(4,2)
            non_empty_iou_test_tensor[:,:] = -1
            non_empty_tensor_index_tensor = torch.zeros(2,2)
            non_empty_tensor_index_tensor[:,:] = -1
        else:
            non_empty_iou_test_tensor = iou_test_tensor[:,:max_num]
            non_empty_tensor_index_tensor = tensor_index_tensor[:max_num,:]

        return predictions_dicts, non_empty_iou_test_tensor, non_empty_tensor_index_tensor

    def process_anno(self, anno, class_name=['Car']):
        ind = np.where(anno['name'] == class_name[0])
        loc = anno['location'][ind]
        dim = anno['dimensions'][ind]
        rot = anno['rotation_y'][ind]
        if len(loc) == 0:
            d3_box = []
        else:
            d3_box = np.concatenate((loc,dim,rot.reshape(-1,1)),axis=1)
        return np.array(d3_box)


In [6]:
_2d_path = './data/clocs_data/2D'
_3d_path = './data/clocs_data/3D'
input_data = './data/clocs_data/input_data'
train_ind_path = './index/trainval.txt'
infopath = './data/clocs_data/info/kitti_infos_trainval.pkl'

train_dataset = clocs_data(_2d_path, _3d_path,train_ind_path, input_data, infopath)

FileNotFoundError: [Errno 2] No such file or directory: './index/trainval.txt'

In [ ]:
train_data = DataLoader(
    train_dataset,
    batch_size=1,
    num_workers=8,
    pin_memory=True
)



In [ ]:
count = 0

for fusion_input,tensor_index,positives,negatives,one_hot_targets,label_n,idx in (train_data):
    print("fusion_input\n",fusion_input)
    print(fusion_input.shape)

    print("tensor_index\n",tensor_index)
    print(tensor_index.shape)

    print("positives\n",positives)
    print(positives.shape)
    
    print("negatives\n",negatives)
    print(negatives.shape)

    print("one_hot_targets\n",one_hot_targets)
    print(one_hot_targets.shape)

    print("label_n\n",label_n)
    print(label_n.shape)

    print("idx\n",idx)

    count += 1

    if count == 7 : break

fusion_input
 tensor([[[[ 2.4300e-02,  5.5168e-02,  1.0081e-01,  6.1437e-03,  2.5525e-02,
            7.2263e-02,  6.0891e-02, -1.0000e+01, -1.0000e+01]],

         [[ 1.0937e-01,  1.0937e-01,  1.5447e-01,  1.5447e-01,  1.5447e-01,
            1.0937e-01,  1.0937e-01,  1.5447e-01,  1.0937e-01]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00, -1.0000e+01, -1.0000e+01]],

         [[ 8.4841e-02,  8.4841e-02,  7.8425e-02,  7.8425e-02,  7.8425e-02,
            8.4841e-02,  8.4841e-02,  7.8425e-02,  8.4841e-02]]]])
torch.Size([1, 4, 1, 9])
tensor_index
 tensor([[[17,  1],
         [19,  1],
         [20,  0],
         [23,  0],
         [25,  0],
         [28,  1],
         [29,  1],
         [32,  0],
         [32,  1]]])
torch.Size([1, 9, 2])
positives
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([1, 20000])
negatives
 tensor([[1., 1., 1.,  ..., 1., 1., 1.]])
torch.Size([1, 20000])
one_hot_targets
 tensor([[[0.],
 

In [ ]:
data = torch.load("./data/clocs_data/input_data/000007.pt")
data

{'input_data': {'fusion_input': array([[ 3.4720402e-02,  6.5608583e-03,  2.2248727e-04,  1.5097714e-04,
           6.7675374e-03, -1.0000000e+01],
         [ 1.5373568e-01,  1.1602955e-01,  1.1602955e-01,  1.1602955e-01,
           1.5373568e-01,  1.1602955e-01],
         [ 9.9980003e-01,  9.9919999e-01,  1.3300000e-01,  5.0000002e-04,
           0.0000000e+00, -1.0000000e+01],
         [ 8.3436064e-02,  8.4934637e-02,  8.4934637e-02,  8.4934637e-02,
           8.3436064e-02,  8.4934637e-02]], dtype=float32),
  'tensor_index': array([[ 0,  0],
         [ 1,  1],
         [ 2,  1],
         [16,  1],
         [25,  0],
         [25,  1]])},
 'label': {'target_for_fusion': tensor([[[0],
           [0]]]),
  'positive_index': tensor([[0, 0]]),
  'negative_index': tensor([[1, 1]]),
  'label_n': 3}}

In [ ]:
negative_cls_weights = negatives.type(torch.float32) * 1.0
cls_weights = negative_cls_weights + 1.0 * positives.type(torch.float32)
pos_normalizer = positives.sum(1, keepdim=True).type(torch.float32)
cls_weights /= torch.clamp(pos_normalizer, min=1.0)

In [ ]:
cls_weights

tensor([[1., 1., 1.,  ..., 1., 1., 1.]])

In [ ]:
negatives.sum(1, keepdim=True).type(torch.float32)

tensor([[20000.]])

In [ ]:
a = tensor_index.reshape(-1,2).size()


tensor([[0, 5]])

In [ ]:
out_1 = torch.zeros(1,200,20000)
out_1[:,:,:] = -9999999

x = torch.randn(1,1,1,92403)

In [ ]:
out_1[:,tensor_index[:,0],tensor_index[:,1]] = x[0,:,0,:]

RuntimeError: shape mismatch: value tensor of shape [92403] cannot be broadcast to indexing result of shape [1, 1, 2]

In [8]:
import torch

a = torch.load("/media/drcl/DATADRIVE1/kitti/clocs/data/clocs_data/input_data/000189.pt")
b = torch.load("./data/clocs_data/test/000189.pt")

a["input_data"]["fusion_input"].shape

(4, 23)

In [9]:
b["input_data"]["fusion_input"].shape

(5, 23)

In [ ]:
score_array = np.array([[1,1,1,1,1,1]])

In [ ]:
data['input_data']['fusion_input']

array([[ 3.4720402e-02,  6.5608583e-03,  2.2248727e-04,  1.5097714e-04,
         6.7675374e-03, -1.0000000e+01],
       [ 1.5373568e-01,  1.1602955e-01,  1.1602955e-01,  1.1602955e-01,
         1.5373568e-01,  1.1602955e-01],
       [ 9.9980003e-01,  9.9919999e-01,  1.3300000e-01,  5.0000002e-04,
         0.0000000e+00, -1.0000000e+01],
       [ 8.3436064e-02,  8.4934637e-02,  8.4934637e-02,  8.4934637e-02,
         8.3436064e-02,  8.4934637e-02]], dtype=float32)

In [ ]:
data['input_data']['fusion_input'] = np.vstack([data['input_data']['fusion_input'],score_array])

In [ ]:
data['input_data']['fusion_input'].shape

(5, 6)

In [ ]:
int("000003")

3

In [ ]:
data['input_data']["tensor_index"]

array([[ 0,  0],
       [ 1,  1],
       [ 2,  1],
       [16,  1],
       [25,  0],
       [25,  1]])

In [13]:
import pickle


with open('/media/drcl/DATADRIVE1/kitti/clocs/data/clocs_data/3D_box_points/000304.pickle', 'rb') as f:
    data = pickle.load(f)

In [15]:
data

[]

In [18]:
b = torch.load("/media/drcl/DATADRIVE1/kitti/clocs/data/clocs_data/3D/000304.pt")
b = torch.load("/media/drcl/DATADRIVE1/kitti/clocs/data/clocs_data/input_data/000304.pt")


In [19]:
b['input_data']

{'input_data': {'fusion_input': array([[-1., -1.],
         [-1., -1.],
         [-1., -1.],
         [-1., -1.]], dtype=float32),
  'tensor_index': array([[-1., -1.],
         [-1., -1.]], dtype=float32)},
 'label': {'target_for_fusion': tensor([], size=(1, 0, 1), dtype=torch.int64),
  'positive_index': tensor([], size=(1, 0), dtype=torch.int64),
  'negative_index': tensor([], size=(1, 0), dtype=torch.int64),
  'label_n': 1}}

In [13]:
b = torch.load("./log/5-Adadelta/faster/result/10.pt")

In [14]:
len(b)

3769

In [15]:
b[1]

{'name': array(['Car', 'Car'], dtype='<U3'),
 'truncated': array([0., 0.]),
 'occluded': array([0., 0.]),
 'alpha': array([-4.8127623, -4.550305 ], dtype=float32),
 'bbox': array([[656.94366, 188.31335, 701.9653 , 223.98334],
        [657.2818 , 192.56345, 700.6856 , 222.95459]], dtype=float32),
 'dimensions': array([[4.0677466, 1.5192277, 1.7264566],
        [4.2963185, 1.4224142, 1.6721627]], dtype=float32),
 'location': array([[ 3.2207198,  2.3027563, 34.54227  ],
        [ 3.7493799,  2.550825 , 39.0307   ]], dtype=float32),
 'rotation_y': array([-4.7213397, -4.4560156], dtype=float32),
 'score': array([0.71337813, 0.2413632 ], dtype=float32),
 'boxes_lidar': array([[34.838104 , -3.1938841, -2.0477982,  4.0677466,  1.7264566,
          1.5192277,  3.1505432],
        [39.32901  , -3.7193356, -2.254514 ,  4.2963185,  1.6721627,
          1.4224142,  2.885219 ]], dtype=float32),
 'frame_id': '000002'}

In [ ]:
with open("/media/drcl/DATADRIVE1/kitti/clocs/data/clocs_data/info/data.pickle","rb") as fr:
    data = pickle.load(fr)
print(data)